In [2]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt

# Verify that all libraries loaded correctly
print(f"Pandas Version: {pd.__version__}")
print(f"GeoPandas Version: {gpd.__version__}")
print(f"OSMnx Version: {ox.__version__}")

Pandas Version: 2.2.3
GeoPandas Version: 1.1.1
OSMnx Version: 2.0.6


In [3]:
# 1. Define the place and network type
place_name = "Austin, Texas, USA"
network_type = "walk" # Focus on paths pedestrians can use

In [4]:
# 2. Download the street network graph for the defined area
# This will download all walking paths from OpenStreetMap and model it as a graph
print(f"Downloading {network_type} network for {place_name}...")
G = ox.graph_from_place(
    place_name,
    network_type=network_type,
    # Consolidate nodes that are close together (e.g., at divided roads)
    # to represent "true" intersections more accurately
    retain_all=False, # Only keep the largest connected components
    simplify=True
)

print(f"Graph loaded. Nodes: {len(G.nodes):,}, Edges: {len(G.edges):,}")

Graph loaded. Nodes: 161,414, Edges: 465,486


In [6]:
# 3. Project the graph for accurate area and distance calculations (using local UTM zone)
# The graph is projected automatically to a suitable local projection (in meters)
print("Projecting graph...")
G_proj = ox.project_graph(G, to_latlong=False)

Projecting graph...


In [8]:
# 4. Get the total area of the place boundary
# We use the boundary geometry to normalize the intersection count
city_boundary = ox.geocode_to_gdf(place_name)
# Convert to a local CRS to get area in square meters, then convert to square kilometers
target_crs = ox.projection.utm_crs(city_boundary.iloc[0].geometry.centroid.y, city_boundary.iloc[0].geometry.centroid.x)
city_area_sq_km = ox.project_gdf(city_boundary).area.iloc[0] / 10**6

AttributeError: module 'osmnx.projection' has no attribute 'utm_crs'

In [ ]:
# 5. Calculate the Intersection Density
# Nodes with degree > 1 are considered true intersections (excluding dead-ends)
